# Libraries

In [ ]:
# Importing libraries

from torch.utils.data import DataLoader, TensorDataset
from torch.linalg import matrix_norm
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import datetime
import random
import meshio
import torch

import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import LinearLocator
from utilities import *
from scipy import spatial

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(0)
np.random.seed(0)

# Loading data

In [ ]:
# General variables

x, y, z = 43, 43, 33
nsteps  = 34
sim     = 8
d_in    = 5
d_out   = 1
batch   = 10
width   = 6
f_eval  = 15
l_rate  = 0.05
mini    = 0


In [ ]:
# read data 6 per simulation

velocities = random.sample( range(1, 243), 90 )
sim = len(velocities) * 6
rnd = np.arange(0, sim)
np.random.shuffle( rnd )
eighty = int( sim * 0.8 )
ntrain = rnd[:eighty]
neval  = rnd[eighty:]

print( f"Training files IDs   : {len(ntrain)}" )
print( f"Evaluation files IDs : {len(neval)}" )
print( f"\n\t===============================================\n" )

x, y, z = 43, 43, 33
nsteps  = 25
d_in    = 5
d_out   = 1
step    = 0
data    = np.ones( [len(velocities), 1, nsteps, x, y, z] )
x_data  = np.ones( [sim, 1, d_in,  x, y, z] )
y_data  = np.ones( [sim, 1, d_out, x, y, z] )

for i, j in enumerate( velocities ):
    time = 0
    for k in range( 25 ):
        if k == 0: print(f"\t{i+1} - { len(velocities) }" )
        file = "./data/vXYZ" + str(j) + "_" + str(k) + ".vtk"
        rXYZ = meshio.read( file )
        tmp  = clean( rXYZ.point_data['vel_XYZ'] )
        
        data[ i, :,time,:,:,: ] = np.concatenate( ([tmp.reshape(x, y, z)]) )
        time += 1
        
    for sim_out in range( 0, nsteps-d_in, d_in-1 ):
        x_data[ step,0,:,:,:,: ] = data[ i, 0, sim_out:sim_out+d_in, :,:,: ]
        y_data[ step,0,:,:,:,: ] = data[ i, 0, sim_out+d_in, :,:,: ].reshape( 1,x,y,z )
        step += 1

    x_data[ step,0,:,:,:,: ] = data[ i, 0, nsteps-d_in-1:nsteps-1, :,:,: ]
    y_data[ step,0,:,:,:,: ] = data[ i, 0, nsteps-1, :,:,: ].reshape( 1,x,y,z )
    step += 1
    
    print(f"vtk-{j} loaded\n")

print(f"\t----------------------------------------\n\
        \tAll simulations loaded...\n\
\t----------------------------------------\n")

x_train = torch.from_numpy( x_data[ ntrain,:,:,:,:,: ] )
y_train = torch.from_numpy( y_data[ ntrain,:,:,:,:,: ] )
x_eval  = torch.from_numpy( x_data[ neval, :,:,:,:,: ] )
y_eval  = torch.from_numpy( y_data[ neval, :,:,:,:,: ] )


# FNO

In [ ]:
# 3D with fourier layers, NORMALIZATION 0, 1
################################################################

class SpectralConv3d(nn.Module):
    def __init__(self, in_ch, out_ch, num):
        super(SpectralConv3d, self).__init__()
        self.in_ch  = in_ch
        self.out_ch = out_ch
        self.num = num
        
        self.m1 = 43//self.num
        self.m2 = 43//self.num
        self.m3 = 33//self.num
        
        self.w1    = nn.Parameter( torch.rand(self.out_ch, self.out_ch, self.m1, self.m2, self.m3, dtype=torch.cdouble) )
        self.conv1 = nn.Conv3d( self.in_ch, self.in_ch,  1, dtype=torch.double )
        self.conv2 = nn.Conv3d( self.in_ch, self.out_ch, 1, dtype=torch.double )
        self.relu1 = nn.ReLU()

    def compl_mul3d(self, input, weights):
        return torch.einsum("bixyz,ioxyz->boxyz", input, weights)

    def forward( self, x ):
        x_ft   = torch.fft.fftn( x, dim=(2,3,4) )
        mid_ft = torch.zeros( x.size(), dtype=torch.cdouble, device=x.device )
        mid_ft[:,:, :self.m1,   :self.m2,  :self.m3] = self.compl_mul3d(x_ft[:,:,  :self.m1,  :self.m2,  :self.m3], self.w1)
        x_mid  = torch.fft.ifftn( mid_ft, s=(x.size(-3), x.size(-2), x.size(-1)) ).to( torch.double )
        
        x_out  = self.conv1( x_mid )
        x_out  = self.relu1( x_out )
        x_out  = self.conv2( x_out )
        
        return x_out
        
    
class W(nn.Module):
    def __init__(self, out_channels):
        super(W, self).__init__()
        self.out_ch = out_channels
        
        self.conv1  = nn.Conv3d( self.out_ch, self.out_ch, 1, dtype=torch.double )
        self.conv2  = nn.Conv3d( self.out_ch, self.out_ch, 1, dtype=torch.double )
        self.relu1  = nn.ReLU()
        
    def forward( self, x ):
        
        x_mid1 = self.conv1( x )
        x_mid2 = self.relu1( x_mid1 )
        x_out  = self.conv2( x_mid2 )
        
        return x_out
    

class PQC(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(PQC, self).__init__()
        self.in_ch  = in_ch
        self.mid_ch = mid_ch
        self.out_ch = out_ch
        self.pq1    = nn.Linear( self.in_ch,  self.mid_ch, dtype=torch.double )
        self.pq2    = nn.Linear( self.mid_ch, self.mid_ch, dtype=torch.double )
        self.pq3    = nn.Linear( self.mid_ch, self.out_ch, dtype=torch.double )

    def forward( self, x ):
        x = self.pq1( x )
        x = self.pq2( x )
        x = self.pq3( x )
        
        return x


class FNO3d(nn.Module):
    def __init__(self, d_in, width, d_out ):
        super(FNO3d, self).__init__()
        self.width  = width
        self.input  = d_in
        self.output = d_out

        self.pX     = PQC( self.input, 3, self.width )
        self.qX1    = PQC( self.width, self.width, self.width )
        self.qX2    = nn.Conv3d( self.width, 1, 1, dtype=torch.double )
        
        self.convX1 = SpectralConv3d( self.width, self.width, 3 )
        self.convX2 = SpectralConv3d( self.width, self.width, 3 )
        self.convX3 = SpectralConv3d( self.width, self.width, 3 )
        self.convX4 = SpectralConv3d( self.width, self.width, 3 )
        self.convX5 = SpectralConv3d( self.width, self.width, 3 )
        self.convX6 = SpectralConv3d( self.width, self.width, 3 )
        self.wX1    = W( self.width )
        self.wX2    = W( self.width )
        self.wX3    = W( self.width )
        self.wX4    = W( self.width )
        self.wX5    = W( self.width )
        self.wX6    = W( self.width )
        self.relu1  = nn.ReLU()
        self.relu2  = nn.ReLU()

        self.batch1 = torch.nn.BatchNorm3d(self.width,  dtype=torch.double)
        self.batch2 = torch.nn.BatchNorm3d(self.output, dtype=torch.double)
        
        
    def forward(self, x):
        x_in = x[ :,0,:,:,:,: ].permute( 0,2,3,4,1 )
        x_in = self.pX( x_in ).permute( 0,4,1,2,3 )
        x_in = self.relu1( x_in )
    
        xx1 = self.convX1( x_in.clone() )
        xx2 = x_in.clone()
        xx2 = self.wX1( xx2 )
        xx_mid1 = ( xx1 + xx2 )
        
        xx1 = self.convX2( xx_mid1.clone() )
        xx2 = xx_mid1.clone()
        xx2 = self.wX2( xx2 )
        xx_mid2 = ( xx1 + xx2 )
        
        xx1 = self.convX3( xx_mid2.clone() )
        xx2 = xx_mid2.clone()
        xx2 = self.wX3( xx2 )
        xx_mid3 = ( xx1 + xx2 )
        
        #######################################################
        
        xx1 = self.convX4( xx_mid3.clone() )
        xx2 = xx_mid3.clone()
        xx2 = self.wX4( xx2 )
        xx_mid4 = ( xx1 + xx2 + xx_mid3 )
        
        xx1 = self.convX5( xx_mid4.clone() )
        xx2 = xx_mid4.clone()
        xx2 = self.wX5( xx2 )
        xx_mid5 = ( xx1 + xx2 + xx_mid2 )

        xx1 = self.convX6( xx_mid5.clone() )
        xx2 = xx_mid5.clone()
        xx2 = self.wX6( xx2 )
        xx_mid6 = ( xx1 + xx2 + xx_mid1 )

        x = self.qX1( xx_mid6.permute( 0,2,3,4,1 ) ).permute( 0,4,1,2,3 )
        x = self.relu2( x )
        x = self.qX2( x )
        x = x + x_in[ :,-1,:,:,: ].unsqueeze(1)
        x_out = x.unsqueeze( 1 )
        
        return x_out
        

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end   = torch.cuda.Event(enable_timing=True)
start.record()
torch.manual_seed(0)

# model
try:
    del( model )
except:
    pass

try:
    torch.cuda.empty_cache()
except:
    pass

mini   = 0
batch  = 25
width  = 6
out_w  = 4
epochs = 61
f_eval = 15
l_rate = 0.001
model  = FNO3d( d_in, width, d_out ).cuda()

optimizer    = torch.optim.SGD( model.parameters(), lr=l_rate )
train_loader = DataLoader( TensorDataset(x_train, y_train), batch_size=batch, shuffle=True )
eval_loader  = DataLoader( TensorDataset(x_eval,  y_eval),  batch_size=batch, shuffle=False )
loss_3D        = nn.MSELoss()

total_train_l2 = []
total_eval_l2  = []
loss_all       = []
eval_all       = []
eval_idx       = 0

start_time     = datetime.datetime.now()
start_time     = start_time.strftime( "%c" )
print( f"\n\tStart time: {start_time}" )
for epoch in range( epochs ):
    model.train()
    train_l2 = 0
    l2_all = np.zeros( [out_w] )

    for x_train_loader, y_train_loader in train_loader:
        x_train_loader = x_train_loader.cuda()
        y_train_loader = y_train_loader.cuda().reshape( y_train_loader.shape[0],1,1,x*y*z )
        optimizer.zero_grad()

        train_x_norm = RangeNormalizer( x_train_loader, low=mini, high=1.0 )
        x_train_loader = train_x_norm.encode( x_train_loader )
        out_train  = model( x_train_loader )
        out_train_decode = out_train.reshape( out_train.shape[0],1,1,x*y*z )
        
        loss_l2 = torch.sqrt( loss_3D( out_train_decode, y_train_loader ) )
        loss_l2.backward()
        optimizer.step()
        
        train_l2 += loss_l2.item()
        
    total_train_l2.append( train_l2 / len(ntrain) )
    
    if ( epoch % f_eval != 0 and epoch != 0 ):
        continue

    model.eval()
    eval_l2 = 0.0
    
    with torch.no_grad():
        for x_eval_loader, y_eval_loader in eval_loader:
            x_eval_loader = x_eval_loader.cuda()
            y_eval_loader = y_eval_loader.cuda().reshape( y_eval_loader.shape[0],1,1,x*y*z )
            
            eval_x_norm = RangeNormalizer( x_eval_loader, low=mini, high=1.0 )
            x_eval_loader = eval_x_norm.encode( x_eval_loader )
            out_eval  = model( x_eval_loader )
            out_eval_decode = out_eval.reshape( out_eval.shape[0],1,1,x*y*z )
                     
            evaluation = torch.sqrt( loss_3D( out_eval_decode, y_eval_loader ) )
            eval_l2 += evaluation.item()
    total_eval_l2.append( eval_l2 / len(neval) )    

    print( "\n\tEpoch: {}/{}".format( epoch, epochs-1 ) )
    print( "\tTime: {}".format( datetime.datetime.now().strftime( "%X" ) ) )
    print( "train : {:4.4f}".format( train_l2 / len(ntrain) ) )
    print( "eval  : {:4.4f}\n".format( eval_l2 / len(neval) ) )

end.record()
torch.cuda.synchronize()
print( "\n\tTraining Finished..." )

print( "Elapsed time: {:4.2f}".format(start.elapsed_time(end)/6e4) )
torch.save(model.state_dict(), "./model_saved_30Sep_v03" ) 


In [ ]:
# Error graph

trainx = np.arange( 0, len(total_train_l2) )
evalx  = np.arange( 0, len(total_train_l2), f_eval )
fig, axes = plt.subplots( figsize=(16,6) )
axes.plot( trainx, total_train_l2[:], color="black", label="train loss actual" )
axes.plot( evalx, total_eval_l2[:], color="red", label="eval loss actual" )
plt.title( "Training and evaluation error", fontsize=18 )
plt.xlabel( "Epocas", fontsize=16 )
plt.ylabel( "Error", fontsize=16 )
plt.legend( fontsize=16 )
plt.grid()
plt.show()

np.savetxt( "errorTrain.txt", total_train_l2, fmt='%1.6e' )
np.savetxt( "errorEval.txt",  total_eval_l2,  fmt='%1.6e' )

In [ ]:
# Load a saved model
batch  = 8
width  = 6
out_w  = 4
epochs = 61
f_eval = 15
l_rate = 0.05
model = FNO3d( d_in, width, d_out ).cuda()
model.load_state_dict(torch.load( "./final_saved" ))
model.eval()